<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0


    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-05 10:03:40
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: 41.44 K (0.3%)
Current PnL: -17.75 L (-11.94%)
CY Booked + Current PnL: -7.89 L (-5.31%)
-------------------
Total profit:  3.01 L
Total loss:  -20.76 L
-------------------
Total Booked + Current PnL: 19.09 L (15.69%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.03%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.46 L (55.92%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.98%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.34,71.0,H-MC,7.45,143368.0,28432.0,8158.0,0.39,24.74,5.69,31.83,81.0,3.49,1.08,48.60,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-12.87,65.0,H-LC,10.12,174053.0,17876.0,8215.0,0.15,11.45,4.72,16.71,22.0,2.18,1.31,30.88,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,106.19,67.0,M-SC,6.35,88844.0,-11933.0,12029.0,0.41,-11.84,13.54,0.09,245.0,-0.99,0.67,16.72,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,48.0,M-LC,4.51,214451.0,12972.0,14240.0,-0.02,6.44,6.64,13.51,60.0,0.91,1.61,19.85,XY25,NTT,BANKS
26,DMART,5201.00,-4.91,73.0,H-LC,15.53,237400.0,52919.0,17449.0,0.32,28.69,7.35,38.14,38.0,3.03,1.78,42.33,X40N,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-20.77,40.0,H-MC,2.18,136312.0,524.0,42229.0,0.56,0.39,30.98,31.48,107.0,0.01,1.02,12.55,X40,ATH,INSURANCE
31,HAVELLS,2069.16,-0.51,60.0,H-MC,1.92,246683.0,-1048.0,76102.0,-0.21,-0.42,30.85,30.30,92.0,-0.01,1.85,12.81,X40,ATH,ELECTRICAL


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-0.65,38.0,H-LC,9.89,227085.0,-14839.0,78117.0,0.21,-6.13,34.40,26.15,8.0,-0.19,1.71,7.31,X40,ATH,IT
85,VBL,671.64,-14.17,41.0,H-LC,6.89,308983.0,-6859.0,119515.0,-1.06,-2.17,38.68,35.67,5.0,-0.06,2.32,11.38,X40N,ATH,FMCG
58,RELIANCE,1533.00,-14.79,43.0,H-LC,7.24,213361.0,2995.0,25795.0,0.62,1.42,12.09,13.68,37.0,0.12,1.60,18.18,XY25,NTT,REFINERIES
41,INFY,2275.00,-19.60,44.0,H-LC,10.89,312940.0,52.0,171648.0,0.42,0.02,54.85,54.87,3.0,0.00,2.35,6.67,X40,BTT,IT
45,JIOFIN,387.00,-0.91,44.0,H-LC,14.01,219100.0,1694.0,51795.0,1.15,0.78,23.64,24.61,48.0,0.03,1.65,56.02,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-4.62,57.0,H-LC,1.86,236475.0,-9676.0,88891.0,0.86,-3.93,37.59,32.18,16.0,-0.11,1.78,30.70,X200,ATH,IT
52,NESTLEIND,1377.00,-8.48,61.0,H-LC,2.54,281479.0,16053.0,40730.0,-0.80,6.05,14.47,21.40,11.0,0.39,2.12,13.09,XY25,NTT,FMCG
43,ITC,452.00,-38.34,48.0,H-LC,3.05,199243.0,-895.0,21777.0,-2.03,-0.45,10.93,10.44,4.0,-0.04,1.50,5.09,X40,NTT,FMCG
19,CIPLA,1795.00,-18.75,59.0,H-LC,5.63,216844.0,12344.0,29079.0,0.32,6.04,13.41,20.25,10.0,0.42,1.63,15.30,X40N,BTT,PHARMA
85,VBL,671.64,-14.17,41.0,H-LC,6.89,308983.0,-6859.0,119515.0,-1.06,-2.17,38.68,35.67,5.0,-0.06,2.32,11.38,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,4029.73,80.0,M-SC,9.55,128785.0,-18583.0,70394.0,7.19,-12.61,54.66,35.16,236.0,-0.26,0.97,31.54,XY24,NTT,AUTO
69,SONACOMS,1006.00,-37.09,48.0,M-SC,12.69,78365.0,-22794.0,97682.0,2.25,-22.53,124.65,74.03,202.0,-0.23,0.59,10.50,AR,BTT,AUTO
87,WHIRLPOOL,2270.00,-38.77,65.0,M-SC,4.19,103478.0,11980.0,66774.0,2.25,13.09,64.53,86.07,223.0,0.18,0.78,50.27,XR,NTT,DURABLES
73,TANLA,1963.11,-24.44,63.0,H-SC,8.20,172782.0,-66633.0,339586.0,2.17,-27.83,196.54,114.01,133.0,-0.20,1.30,58.47,AR,ATH,IT
17,CAMS,5211.76,-5.38,50.0,H-SC,5.47,107453.0,5449.0,38479.0,1.94,5.34,35.81,43.06,122.0,0.14,0.81,23.37,X40N,ATH,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SIS,528.00,2186.53,45.0,H-SC,5.70,61515.0,-19575.0,28248.0,-2.16,-24.14,45.92,10.69,156.0,-0.69,0.46,24.67,OX40N,NTT,MISC
43,ITC,452.00,-38.34,48.0,H-LC,3.05,199243.0,-895.0,21777.0,-2.03,-0.45,10.93,10.44,4.0,-0.04,1.50,5.09,X40,NTT,FMCG
85,VBL,671.64,-14.17,41.0,H-LC,6.89,308983.0,-6859.0,119515.0,-1.06,-2.17,38.68,35.67,5.0,-0.06,2.32,11.38,X40N,ATH,FMCG
83,VAIBHAVGBL,521.00,41.60,48.0,H-SC,11.61,94638.0,-57353.0,131481.0,-0.96,-37.73,138.93,48.77,125.0,-0.44,0.71,10.15,XR,NTT,JEWELLERY
13,BERGEPAINT,680.00,-19.16,45.0,H-MC,7.73,217303.0,-10036.0,58107.0,-0.84,-4.41,26.74,21.14,106.0,-0.17,1.63,22.48,XY24,NTT,PAINTS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,106.19,67.0,M-SC,6.35,88844.0,-11933.0,12029.0,0.41,-11.84,13.54,0.09,245.0,-0.99,0.67,16.72,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.97,57.0,H-SC,3.44,220383.0,-49284.0,85619.0,0.67,-18.28,38.85,13.47,138.0,-0.58,1.66,11.47,XY24,NTT,PAINTS
80,UJJIVANSFB,60.0,83.20,44.0,H-SC,13.19,115965.0,-26514.0,46038.0,-0.37,-18.61,39.70,13.70,163.0,-0.58,0.87,36.87,OX40N,NTT,BANKS
39,INDIGOPNTS,1408.0,116.52,52.0,M-SC,3.52,141930.0,-32629.0,32658.0,-0.71,-18.69,23.01,0.02,221.0,-1.00,1.07,23.45,OX40N,NTT,PAINTS
18,CERA,9475.0,-19.81,45.0,H-SC,2.77,114255.0,-29548.0,56293.0,-0.27,-20.55,49.27,18.60,149.0,-0.52,0.86,26.08,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-27.08,48.0,H-LC,14.33,290977.0,-54979.0,178834.0,-0.01,-15.89,61.46,35.80,1.0,-0.31,2.19,3.08,X40,BTT,IT
41,INFY,2275.00,-19.60,44.0,H-LC,10.89,312940.0,52.0,171648.0,0.42,0.02,54.85,54.87,3.0,0.00,2.35,6.67,X40,BTT,IT
43,ITC,452.00,-38.34,48.0,H-LC,3.05,199243.0,-895.0,21777.0,-2.03,-0.45,10.93,10.44,4.0,-0.04,1.50,5.09,X40,NTT,FMCG
85,VBL,671.64,-14.17,41.0,H-LC,6.89,308983.0,-6859.0,119515.0,-1.06,-2.17,38.68,35.67,5.0,-0.06,2.32,11.38,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,46.0,H-LC,13.09,247411.0,-14208.0,133429.0,-0.16,-5.43,53.93,45.57,7.0,-0.11,1.86,5.28,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,0.81,78.0,H-MC,20.36,187285.0,36372.0,20938.0,0.04,24.10,11.18,37.97,93.0,1.74,1.41,55.50,AR,ATH,AUTO
25,DIXON,18940.15,-28.34,71.0,H-MC,7.45,143368.0,28432.0,8158.0,0.39,24.74,5.69,31.83,81.0,3.49,1.08,48.60,X40N,ATH,IT
26,DMART,5201.00,-4.91,73.0,H-LC,15.53,237400.0,52919.0,17449.0,0.32,28.69,7.35,38.14,38.0,3.03,1.78,42.33,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,12.23,62.0,H-MC,3.36,193650.0,28980.0,38982.0,-0.39,17.60,20.13,41.28,89.0,0.74,1.46,26.99,X40N,ATH,AC
86,VOLTAS,1918.49,2.00,64.0,H-MC,4.00,214140.0,22398.0,73643.0,1.78,11.68,34.39,50.08,99.0,0.30,1.61,19.27,XY25,ATH,AC
33,HEROMOTOCO,5949.07,0.81,78.0,H-MC,20.36,187285.0,36372.0,20938.0,0.04,24.10,11.18,37.97,93.0,1.74,1.41,55.50,AR,ATH,AUTO
23,DABUR,735.00,-0.95,68.0,H-MC,4.47,215378.0,18350.0,71269.0,-0.08,9.31,33.09,45.49,102.0,0.26,1.62,23.57,XY24,BTT,FMCG
17,CAMS,5211.76,-5.38,50.0,H-SC,5.47,107453.0,5449.0,38479.0,1.94,5.34,35.81,43.06,122.0,0.14,0.81,23.37,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,0.81,78.0,H-MC,20.36,187285.0,36372.0,20938.0,0.04,24.10,11.18,37.97,93.0,1.74,1.41,55.50,AR,ATH,AUTO
51,NATIONALUM,247.44,-44.49,72.0,H-MC,0.56,110436.0,9932.0,20707.0,0.82,9.88,18.75,30.49,79.0,0.48,0.83,45.49,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-38.77,65.0,M-SC,4.19,103478.0,11980.0,66774.0,2.25,13.09,64.53,86.07,223.0,0.18,0.78,50.27,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.65,45.0,H-SC,12.21,134742.0,11406.0,122359.0,-0.74,9.25,90.81,108.45,119.0,0.09,1.01,33.67,AR,ATH,MISC
88,WIPRO,420.00,-12.89,46.0,M-LC,6.90,152954.0,2009.0,107450.0,0.71,1.33,70.25,72.51,53.0,0.02,1.15,7.43,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,61.0,L-SC,36.91,88589.0,-24960.0,65015.0,-0.17,-21.98,73.39,35.27,268.0,-0.38,0.67,116.82,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-3.50,47.0,H-SC,15.66,89853.0,-11110.0,105640.0,-0.46,-11.00,117.57,93.63,148.0,-0.11,0.68,33.83,SR,ATH,CHEMICALS
50,MASFIN,398.61,-18.25,51.0,H-SC,12.87,93090.0,-4890.0,26493.0,-0.11,-4.99,28.46,22.05,152.0,-0.18,0.70,35.31,XR,ATH,FINANCE
51,NATIONALUM,247.44,-44.49,72.0,H-MC,0.56,110436.0,9932.0,20707.0,0.82,9.88,18.75,30.49,79.0,0.48,0.83,45.49,MH,ATH,METALS
80,UJJIVANSFB,60.00,83.20,44.0,H-SC,13.19,115965.0,-26514.0,46038.0,-0.37,-18.61,39.70,13.70,163.0,-0.58,0.87,36.87,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,61.0,L-SC,36.91,88589.0,-24960.0,65015.0,-0.17,-21.98,73.39,35.27,268.0,-0.38,0.67,116.82,XR,NTT,HOTELS
7,ATULAUTO,844.00,4029.73,80.0,M-SC,9.55,128785.0,-18583.0,70394.0,7.19,-12.61,54.66,35.16,236.0,-0.26,0.97,31.54,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,0.81,78.0,H-MC,20.36,187285.0,36372.0,20938.0,0.04,24.10,11.18,37.97,93.0,1.74,1.41,55.50,AR,ATH,AUTO
57,RELAXO,1176.00,-31.55,62.0,H-SC,4.97,63507.0,-52031.0,87024.0,-0.43,-45.03,137.03,30.28,136.0,-0.60,0.48,24.86,X40N,NTT,FOOTWEAR
26,DMART,5201.00,-4.91,73.0,H-LC,15.53,237400.0,52919.0,17449.0,0.32,28.69,7.35,38.14,38.0,3.03,1.78,42.33,X40N,NTT,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.86
1,25,43.80
2,50,73.90


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.93
LC    36.30
MC    25.81
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.75
X40      15.68
X40N     14.41
XY25     10.10
AR        9.51
XR        9.06
OX40N     7.44
X200      1.78
MH        1.76
X5K       1.44
SR        1.11
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.02
H-MC    22.83
H-SC    22.59
M-SC    13.76
M-LC     7.19
M-MC     2.66
L-SC     1.58
L-LC     1.09
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.95,0.56,33.57
IT,13.52,-14.92,71.90
FINANCE,7.43,-22.22,67.29
BANKS,7.36,-16.34,65.74
PAINTS,6.07,-14.33,39.92
MISC,5.86,-17.84,85.39
ELECTRICAL,5.72,-8.05,67.60
AUTO,4.87,-4.44,49.87
AC,3.80,4.14,28.33


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2772329.0,22
AR,1214296.0,10
XR,1088572.0,13
X40,881009.0,10
X40N,517642.0,11
OX40N,469788.0,10
XY25,409368.0,7
SR,196376.0,2
X5K,126175.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2770854.0,27
M-SC,1418303.0,17
H-LC,1392158.0,16
H-MC,1169188.0,17
M-LC,422272.0,5
M-MC,323275.0,2
L-SC,251100.0,3
L-MC,59501.0,1
L-LC,39696.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,929500.0,6
M-SC,XY24,837870.0,7
H-SC,AR,660875.0,5
H-LC,X40,596410.0,5
H-SC,XR,524971.0,6
H-MC,XY24,455482.0,4
H-LC,AR,373043.0,2
M-MC,XY24,323275.0,2
H-LC,X40N,215494.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
